## Download Dataset from Bucket

In [1]:
# Import database
import pandas as pd
import io
from google.cloud import storage
# Create a client
client = storage.Client()

# Define your bucket name
bucket_name = "dsgt-clef-pan-2024"
# Get the bucket
bucket = client.get_bucket(bucket_name)

# List objects in the bucket
blobs = bucket.list_blobs()
for blob in blobs:
    print(blob.name)

10_Iterations_first_5000_no_wikipedia_Ben_GCN_model.pth
10_Iterations_first_5000_wikipedia_False_Ben_GCN_model.pth
50_Iterations_first_5000_wikipedia_True_Convs_10Ben_GCN_model.pth
550_Iterations_first_5000_wikipedia_False_Ben_GCN_model.pth
Ben_Data/CLS_tokens.json
Ben_Data/Initial_embeddings.json
Ben_Data/No_Wiki2Vec/5000/CLS_tokens.json
Ben_Data/No_Wiki2Vec/5000/Initial_embeddings.json
Ben_Data/No_Wiki2Vec/5000/Targets.json
Ben_Data/No_Wiki2Vec/5000/adj_matrixs.json
Ben_Data/No_Wiki2Vec/5000/entity_sentence.json
Ben_Data/No_Wiki2Vec/5000/length.json
Ben_Data/No_Wiki2Vec/5000/nsp_score.json
Ben_Data/Targets.json
Ben_Data/adj_matrixs.json
Ben_Data/entity_sentence.json
Ben_Data/length.json
Ben_Data/nsp_score.json
JackGraphData/
JackGraphData/test.tar.gz
JackGraphData/test_batched.tar.gz
JackGraphData/train.tar.gz
JackGraphData/train_batched.tar.gz
JackGraphData/val.tar.gz
JackGraphData/val_batched.tar.gz
PAN Data/Artem.pkl
PAN Data/GPT-wiki-intro.csv
PAN Data/GPT.pkl
PAN Data/HFD.pkl
PA

In [2]:
from io import StringIO, BytesIO
import numpy as np
# Specify the path to the file in the bucket
def getCSVFromBucket(csvFilePath):
    blob = bucket.blob(csvFilePath)
    content = blob.download_as_string()
    df = pd.read_csv(StringIO(content.decode('utf-8')))
    return df

# Get the parquet file from the bucket
def getParquetFromBucket(parquetFilePath):
    df = pd.DataFrame()

    blobs = bucket.list_blobs(prefix=parquetFilePath)

    # Iterate over blobs and read Parquet files
    for blob in blobs:
        # Download the file as bytes
        content = blob.download_as_string()
        # Read the Parquet file into a DataFrame
        df_part = pd.read_parquet(BytesIO(content), engine='pyarrow')
        # Concatenate with the existing DataFrame
        df = pd.concat([df, df_part])
    return df

GPT = getCSVFromBucket('PAN Data/GPT-wiki-intro.csv')
GPTcondensed = pd.concat([
    pd.DataFrame({
        'text': GPT['generated_intro'] + GPT['generated_text'],
        'label': 1
        #'source':'ai' # 1 for AI
        #'Type': 'GPT'
    }),
    pd.DataFrame({
        'text': GPT['wiki_intro'],
        'label': 0
        #'source': 'human',
        #'Type': 'GPT'
    })
], ignore_index=True)

Nabeel = getCSVFromBucket('PAN Data/HF_NabeelShar_ai_and_human_text.csv')
#rename 'generated' as label
Nabeel = Nabeel.rename(columns={'generated':'label'})
#Nabeel['source'] = Nabeel['generated'].map({0:'human', 1:'ai'})
#Nabeel['Type'] = 'HF Nabeel'

Artem = getParquetFromBucket('PAN Data/HF_artem9k_ai_text_detection_pile/')
#print(Artem.head())
Artem['label'] = Artem['source'].map({'human':0, 'ai':1})
#map source to label
#Artem['Type'] = 'HF_artem9k'

Dmitva = getCSVFromBucket('PAN Data/HF_dmitva_human_ai_generated_text.csv')
Dmitva = pd.DataFrame(pd.concat([Dmitva['human_text'], Dmitva['ai_text']], axis=0), columns=['text'])
Dmitva['label'] = np.hstack(([0]*500000,[1]*500000))

#Dmitva['source'] = np.hstack((['human']*500000,['ai']*500000))
#Dmitva['Type'] = 'HF_dmitva'

# Concatenate all DataFrames
#Final = pd.concat([Dmitva[['source','text','Type']],
#                   Nabeel[['source','text','Type']],
#                   GPTcondensed[['source','text','Type']],
#                   Artem[['source','text','Type']]], axis=0)

Final = pd.concat([Dmitva[['label','text']],
                   Nabeel[['label','text']],
                   GPTcondensed[['label','text']],
                   Artem[['label','text']]], axis=0)

del GPT, GPTcondensed, Nabeel, Artem, Dmitva


In [3]:
print(Final.head())

   label                                               text
0      0  If they get classes at home they don't learnin...
1      0  Students would be happy of having a day off, a...
2      0  If students get involve in many of them, then ...
3      0  Emerson gave us a great point with this statem...
4      0  I disagree and I believe that other humans mak...


In [4]:
from datasets import load_dataset, Dataset, DatasetDict

df = Dataset.from_pandas(Final)
print(df)
del Final

/home/abindal/pan-2024/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset({
    features: ['label', 'text', '__index_level_0__'],
    num_rows: 2738768
})


In [6]:
trainTestValid = df.train_test_split(test_size=0.2, shuffle=True)
testValid = trainTestValid['test'].train_test_split(test_size=0.5, shuffle=True)
trainTestValid_dataset = DatasetDict({
    'train': trainTestValid['train'], 
    'test': testValid['test'], 
    'valid': testValid['train']})

del df, trainTestValid, testValid

In [7]:
print(trainTestValid_dataset)

DatasetDict({
    train: Dataset({
        features: ['label', 'text', '__index_level_0__'],
        num_rows: 2191014
    })
    test: Dataset({
        features: ['label', 'text', '__index_level_0__'],
        num_rows: 273877
    })
    valid: Dataset({
        features: ['label', 'text', '__index_level_0__'],
        num_rows: 273877
    })
})


In [8]:
from transformers import RobertaTokenizerFast
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')

def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True)

tokenized_datasets = trainTestValid_dataset.map(tokenize_function, batched=True)

Map: 100%|██████████| 273877/273877 [01:52<00:00, 2435.16 examples/s]


In [9]:
from peft import LoraConfig, TaskType

lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS, r=1, lora_alpha=1, lora_dropout=0.1
)

/home/abindal/pan-2024/.venv/lib/python3.10/site-packages/torch/cuda/__init__.py:628: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [10]:
# Get roberta model for classification from Hugging Face
from transformers import RobertaForSequenceClassification

model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
from peft import get_peft_model
model = get_peft_model(model, lora_config)

In [12]:
# Print trainable parameters
print(model.print_trainable_parameters())

trainable params: 628,994 || all params: 125,276,164 || trainable%: 0.5020859355176297
None


In [13]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="train/LoRA/", 
    evaluation_strategy="epoch",
    num_train_epochs=2,
    learning_rate=1e-3,
    per_device_train_batch_size=24,
    per_device_eval_batch_size=24,
    weight_decay=0.01,
    save_strategy="epoch",
    load_best_model_at_end=True,
    )

In [14]:
!pip install evaluate
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [15]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [16]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset = tokenized_datasets['train'].shuffle(seed=42).select(range(5000)),
    eval_dataset = tokenized_datasets['valid'].shuffle(seed=42).select(range(5000)),
    compute_metrics=compute_metrics,
)

In [17]:
trainer.train()

Checkpoint destination directory train/LoRA/checkpoint-209 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory train/LoRA/checkpoint-418 already exists and is non-empty.Saving will proceed but saved results may be invalid.


TrainOutput(global_step=418, training_loss=0.19093978005733217, metrics={'train_runtime': 10338.7869, 'train_samples_per_second': 0.967, 'train_steps_per_second': 0.04, 'total_flos': 2650433249280000.0, 'train_loss': 0.19093978005733217, 'epoch': 2.0})

In [18]:
model.save_pretrained("train/LoRAFullDataset/")

In [ ]:
#Load pretrained model
from transformers import RobertaForSequenceClassification
model = RobertaForSequenceClassification.from_pretrained("train/LoRAFullDataset/")
model.eval()

# Evaluate the model
results = trainer.evaluate()
print(results)


In [19]:
# Run inference on the test dataset
predictions = trainer.predict(tokenized_datasets['test'].shuffle(seed=42))
print(predictions)
#Check the accuracy of predictions
print(predictions.metrics)

KeyboardInterrupt: 

### Upload dataset into cloud

In [24]:
trainTestValid_dataset.save_to_disk('./PAN Data/trainTestValid_dataset/')


Saving the dataset (2/2 shards): 100%|██████████| 273877/273877 [00:01<00:00, 146156.20 examples/s]
